In [93]:
import pandas as pd
import requests
import xml.etree.ElementTree as ET
from pathlib import Path

apiKey = "GmEPb1B.bfqJLIhcGAsH9fTJevTglhFpCoZyAAAdhp"

def bundestag_request(resourcetype, id):
    method = "get"
    url = f'https://search.dip.bundestag.de/api/v1/{resourcetype}'
    if id is not None:
        url += "/"+str(id)
    headers = {
        'Content-type': 'application/xml',
        'Authorization': f'ApiKey {apiKey}',
    }
    rsp = requests.request(method, url, headers=headers, auth=None)
    return rsp.json()

def bundestag_xml(resourcetype, id):
    file = Path(f'../0_datasets/bundestag/{resourcetype}_{id}.xml')
    if not file.exists(): #fetch file from bundestag api if not already downloaded
        try:
            xmlurl = bundestag_request(resourcetype, id)['fundstelle']['xml_url']
        except:
            print("No xml for id " + str(id))
            return None
        rsp = requests.request("get", xmlurl)
        with open(file, 'wb') as f:
            f.write(rsp.content)

    tree = ET.parse(file)
    root = tree.getroot()
    return root

def bundestag_search(resourcetype, wahlperiode, datestart = "1900-01-01", dateend = "2900-01-01", cursor = None):
    print(cursor)
    method = "get"
    url = f'https://search.dip.bundestag.de/api/v1/{resourcetype}?f.zuordnung=BT&f.datum.start={datestart}&f.datum.end={dateend}&f.wahlperiode={wahlperiode}'
    if(cursor is not None):
        url = url + f'&cursor={cursor}'

    headers = {
        'Content-type': 'application/xml',
        'Authorization': f'ApiKey {apiKey}',
    }
    rsp = requests.request(method, url, headers=headers, auth=None)
    data = rsp.json()

    ids=[]
    for p_id in rsp.json()["documents"]:
        ids.append((p_id["id"], p_id["datum"]))

    if len(ids) > 0:
        ids.append(bundestag_search(resourcetype, wahlperiode, datestart, dateend, rsp.json()["cursor"]))

    return ids

ids = bundestag_search("plenarprotokoll","19")

for id in ids:
    bundestag_xml("plenarprotokoll", id[0])

None
AoJwuNGA1-8CNFBsZW5hcnByb3Rva29sbC01MzAy
AoJw0OW4geQCNFBsZW5hcnByb3Rva29sbC00NzQ4
AoJw0OL0pt8CM1BsZW5hcnByb3Rva29sbC05MDg=
No xml for id ('5301', '2020-01-15')
